##### Copyright 2018 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Reescribir automáticamente los símbolos de TF 1.x y API compat.v1

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/guide/migrate/upgrade"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/guide/migrate/upgrade.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a></td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/guide/migrate/upgrade.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver código fuente en GitHub</a>
</td>
  <td>     <a target="_blank" href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/guide/migrate/upgrade.ipynb">     <img src="https://www.tensorflow.org/images/download_logo_32px.png">     Descargar bloc de notas</a>
</td>
</table>


TensorFlow 2.x incluye muchos cambios en la API con respecto a TF 1.x y la API `tf.compat.v1`, como la reordenación de argumentos, el cambio de nombre de los símbolos y la modificación de los valores predeterminados de los parámetros. Realizar manualmente todas estas modificaciones sería tedioso y propenso a errores. Para agilizar los cambios, y para que su transición a TF 2.x sea lo más fluida posible, el equipo de TensorFlow ha creado la utilidad `tf_upgrade_v2` para ayudar a la transición del código heredado a la nueva API.

Nota: `tf_upgrade_v2` se instala automáticamente para TensorFlow 1.13 y posteriores (incluidas todas las compilaciones de TF 2.x).

El uso típico es el siguiente:

<pre class="devsite-terminal devsite-click-to-copy prettyprint lang-bsh">tf_upgrade_v2 \
  --intree my_project/ \
  --outtree my_project_v2/ \
  --reportfile report.txt
</pre>

Acelerará su proceso de actualización convirtiendo los scripts Python existentes de TensorFlow 1.x a TensorFlow 2.x.

El script de conversión automatiza muchas transformaciones mecánicas de las API, aunque muchas API no pueden migrarse de este modo automáticamente. Tampoco es capaz de hacer que su código sea totalmente compatible con los comportamientos y las API de TF2. Por lo tanto, es sólo una parte de su proceso de migración.

## Módulos de compatibilidad

Algunos símbolos de la API no pueden actualizarse simplemente reemplazando la cadena de caracteres. Los que no puedan actualizarse automáticamente se mapearán en sus ubicaciones en el módulo `compat.v1`. Este módulo reemplaza símbolos TF 1.x como `tf.foo` por la referencia equivalente `tf.compat.v1.foo`. Si ya está usando las APIs `compat.v1` mediante la importación de TF a través de `import tensorflow.compat.v1 as tf`, el script `tf_upgrade_v2` intentará convertir estos usos a las APIs que no son compat siempre que sea posible. Tenga en cuenta que aunque algunas APIs `compat.v1` son compatibles con los comportamientos de TF2.x, muchas no lo son. Por lo tanto, se recomienda revisar manualmente los reemplazos y migrarlos a las nuevas APIs en el namespace `tf.*` en lugar de `tf.compat.v1` lo antes posible.

Debido a los módulos obsoletos de TensorFlow 2.x (por ejemplo, `tf.flags` y `tf.contrib`), algunos cambios no se pueden solucionar cambiando a `compat.v1`. La actualización de este código puede requerir usar una librería adicional (por ejemplo, [`absl.flags`](https://github.com/abseil/abseil-py)) o cambiar a un paquete en [tensorflow/addons](http://www.github.com/tensorflow/addons).


## Proceso de actualización recomendado

El resto de esta guía muestra cómo usar el script de reescritura de símbolos. Aunque el script es fácil de usar, se recomienda encarecidamente que lo use como parte del siguiente proceso:

1. **Pruebas de unidad**: Asegúrese de que el código que está actualizando tiene un conjunto de pruebas de unidad con una cobertura razonable. Se trata de código Python, por lo que el lenguaje no le protegerá de muchas clases de errores. Asegúrese también de que cualquier dependencia que tenga ya ha sido actualizada para ser compatible con TensorFlow 2.x.

2. **Instale TensorFlow 1.15**: Actualice su TensorFlow a la última versión de TensorFlow 1.x, como mínimo a la 1.15. Ésta incluye la API final de TensorFlow 2.0 en `tf.compat.v2`.

3. **Pruebe con 1.15**: Asegúrese de que sus pruebas de unidad superan este punto. Las ejecutará repetidamente mientras actualiza, por lo que empezar bien es importante.

4. **Ejecute el script de actualización**: Ejecute `tf_upgrade_v2` en todo su árbol de código fuente, incluidas las pruebas. Esto actualizará su código a un formato en el que sólo use símbolos disponibles en TensorFlow 2.0. Se accederá a los símbolos obsoletos con `tf.compat.v1`. Estos eventualmente requerirán atención manual, pero no inmediatamente.

5. **Ejecute las pruebas convertidas con TensorFlow 1.15**: Su código debería seguir funcionando correctamente en TensorFlow 1.15. Vuelva a ejecutar sus pruebas de unidad. Cualquier error en sus pruebas aquí significa que hay un error en el script de actualización. [Por favor, háganoslo saber](https://github.com/tensorflow/tensorflow/issues).

6. **Revise el informe de actualización en busca de advertencias y errores**: El script escribe un archivo de informe que explica cualquier conversión que deba volver a verificar, o cualquier acción manual que deba realizar. Por ejemplo: Cualquier instancia restante de contrib requerirá ser eliminada manualmente. Consulte [el RFC para conocer más detalles](https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md).

7. **Instale TensorFlow 2.x**: En este punto debería ser seguro cambiar a los binarios de TensorFlow 2.x, incluso si se está ejecutando con comportamientos heredados

8. **Haga pruebas con `v1.disable_v2_behavior`**: Si vuelve a ejecutar sus pruebas con un `v1.disable_v2_behavior()` en la función principal de las pruebas, debería obtener los mismos resultados que si las ejecutara con la versión 1.15.

9. **Habilitar comportamiento V2**: Ahora que sus pruebas funcionan usando los binarios TF2, ya puede empezar a migrar su código para evitar `tf.estimator`s y usar sólo los comportamientos TF2 soportados (sin desactivar el comportamiento TF2). Consulte las [Guías de migración](https://tensorflow.org/guide/migrate) para más detalles.

## Usar el script de reescritura de símbolos `tf_upgrade_v2`


### Preparación

Antes de empezar, asegúrese de que TensorFlow 2.x está instalado.

In [ ]:
import tensorflow as tf

print(tf.__version__)

Clone el repositorio git [tensorflow/models](https://github.com/tensorflow/models) para tener algo de código con el que hacer pruebas:

In [ ]:
!git clone --branch r1.13.0 --depth 1 https://github.com/tensorflow/models

### Lea la ayuda

El script debe instalarse con TensorFlow. Aquí está la ayuda incorporada:

In [ ]:
!tf_upgrade_v2 -h

### Ejemplo de código TF1

Aquí tiene un sencillo script TensorFlow 1.0:

In [ ]:
!head -n 65 models/samples/cookbook/regression/custom_regression.py | tail -n 10

Con TensorFlow 2.x instalado no se ejecuta:

In [ ]:
!(cd models/samples/cookbook/regression && python custom_regression.py)

### Archivo individual

El script puede ejecutarse en un único archivo Python:

In [ ]:
!tf_upgrade_v2 \
  --infile models/samples/cookbook/regression/custom_regression.py \
  --outfile /tmp/custom_regression_v2.py

El script mostrará errores si no puede encontrar una solución para el código. 

### Árbol de directorios

Los proyectos habituales, incluido este sencillo ejemplo, usarán mucho más que un archivo. Normalmente se desea actualizar un paquete entero, por lo que el script también puede ejecutarse en un árbol de directorios:

In [ ]:
# update the .py files and copy all the other files to the outtree
!tf_upgrade_v2 \
    --intree models/samples/cookbook/regression/ \
    --outtree regression_v2/ \
    --reportfile tree_report.txt

Tenga en cuenta la única advertencia sobre la función `dataset.make_one_shot_iterator`.

Ahora el script funciona con TensorFlow 2.x:

Tenga en cuenta que, dado que el módulo `tf.compat.v1` está incluido en TF 1.15, el script convertido también se ejecutará en TensorFlow 1.15.

In [ ]:
!(cd regression_v2 && python custom_regression.py 2>&1) | tail

## Informe detallado

El script también informa de una lista de cambios detallados. En este ejemplo encontró una transformación posiblemente insegura e incluyó una advertencia en la parte superior del archivo: 

In [ ]:
!head -n 20 tree_report.txt

Observe de nuevo la única advertencia sobre la función `Dataset.make_one_shot_iterator`.

En otros casos, la salida explicará el razonamiento de los cambios no triviales:

In [ ]:
%%writefile dropout.py
import tensorflow as tf

d = tf.nn.dropout(tf.range(10), 0.2)
z = tf.zeros_like(d, optimize=False)

In [ ]:
!tf_upgrade_v2 \
  --infile dropout.py \
  --outfile dropout_v2.py \
  --reportfile dropout_report.txt > /dev/null

In [ ]:
!cat dropout_report.txt

Aquí está el contenido del archivo modificado, observe cómo el script añade nombres de argumentos para lidiar con los argumentos movidos y renombrados:

In [ ]:
!cat dropout_v2.py

Un proyecto más grande puede contener algunos errores. Por ejemplo, convertir el modelo deeplab:

In [ ]:
!tf_upgrade_v2 \
    --intree models/research/deeplab \
    --outtree deeplab_v2 \
    --reportfile deeplab_report.txt > /dev/null

Produjo los archivos de salida:

In [ ]:
!ls deeplab_v2

Pero hubo errores. El informe le ayudará a determinar lo que debe corregir antes de que se ejecute. Aquí están los tres primeros errores:

In [ ]:
!cat deeplab_report.txt | grep -i models/research/deeplab | grep -i error | head -n 3

## Modo de "seguridad"

El script de conversión también tiene un modo menos invasivo `SAFETY` que simplemente cambia las importaciones para usar el módulo `tensorflow.compat.v1`:

In [ ]:
!cat dropout.py

In [ ]:
!tf_upgrade_v2 --mode SAFETY --infile dropout.py --outfile dropout_v2_safe.py > /dev/null

In [ ]:
!cat dropout_v2_safe.py

Como puede ver, esto no actualiza su código, pero permite que el código TensorFlow 1 se ejecute contra binarios TensorFlow 2. ¡Tenga en cuenta que esto no significa que su código esté ejecutando comportamientos TF 2.x soportados!

## Precauciones

- No actualice partes de su código manualmente antes de ejecutar este script. En particular, las funciones que han sufrido una reordenación de argumentos como `tf.math.argmax` o `tf.batch_to_space` hacen que el script añada incorrectamente argumentos de palabras clave que desajustan su código existente.

- El script asume que `tensorflow` se importa usando `import tensorflow as tf`, o `import tensorflow.compat.v1 as tf`.

- Este script no reordena los argumentos. En su lugar, el script añade argumentos de palabra clave a las funciones que tienen sus argumentos reordenados.

- Consulte [tf2up.ml](https://github.com/lc0/tf2up) para encontrar una herramienta práctica para actualizar blocs de notas Jupyter y archivos Python en un repositorio GitHub.

Para informar de errores en el script de actualización o para solicitar funciones, presente un incidente en [GitHub](https://github.com/tensorflow/tensorflow/issues).